## Data Cleansing을 해보자!
0. 환경설정, 데이터 불러오기
1. 크롤링으로 G2P 제거

### 0. 데이터 불러오기, 환경 설정

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [2]:
import matplotlib.pyplot as plt

import matplotlib.font_manager
font_list = matplotlib.font_manager.findSystemFonts(fontpaths=None, fontext='ttf')

import matplotlib.pyplot as plt
plt.rc('font', family='NanumGothic')

import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False

In [3]:
import transformers
from datasets import load_dataset

##### 데이터 불러오기

In [4]:
# train = load_dataset('Smoked-Salmon-s/TC_Competition',
#                         split='train',
#                         column_names=['ID', 'text', 'target', 'url', 'date'],
#                         revision='280245a63e4e68a2751005a88314959a9dce1c44')
# dataset_train = train.to_pandas().iloc[1:].reset_index(drop=True).astype({'target': 'int64'})

# valid = load_dataset('Smoked-Salmon-s/TC_Competition',
#                         split='validation',
#                         column_names=['ID', 'text', 'target', 'url', 'date'],
#                         revision='280245a63e4e68a2751005a88314959a9dce1c44')
# dataset_valid = valid.to_pandas().iloc[1:].reset_index(drop=True).astype({'target': 'int64'})

In [5]:
all_train_csv = pd.read_csv('/opt/ml/data/train.csv')

### G2P 제거

#### 크롤링을 해보자!

In [6]:
from bs4 import BeautifulSoup
import requests
import re
from tqdm import tqdm
tqdm.pandas()

In [7]:
def title_crawling(url) :
    news = requests.get(url, headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"})
    news_html = BeautifulSoup(news.text,"html.parser")
    
    if 'sports' in url :
        title = news_html.select_one("h4.title")
    
    else :
        title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
        if title == None:
            title = news_html.select_one("#content > div.end_ct > div > h2")
    
    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))

    pattern2 = r'[,\[\]\(\)\"\'\-~\?+/:]'
    title = re.sub(pattern = pattern2, repl='', string=title)

    title = title.replace('&lt;','')
    title = title.replace('&gt;','')
    title = title.replace('&amp;','')
    
    return title

In [9]:
for i in range(20000,20003) :
    title = title_crawling(all_train_csv.loc[i]['url'])
    print(all_train_csv.loc[i]['text'])
    print(title)
    print('================================================')

몸집 큰 육식 공룡은 뿔 난 경우 많아
몸집 큰 육식 공룡은 뿔 난 경우 많아
文 국민의당 與 장기집권 도와…호남민 염원 짓밟는일종합
文 국민의당 與 장기집권 도와…호남민 염원 짓밟는일종합
출마선언 마치고 질문에 답하는 송영길
출마선언 마치고 질문에 답하는 송영길


In [15]:
title_crawling('https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=228&oid=001&aid=0011280429')

'백두산 폭발은 영화적 상상력…언제든 가능한 과학적 현실'

In [ ]:
# 45,678건 처리하는데 약 3시간 정도 걸립니다.

title = all_train_csv['url'].progress_apply(title_crawling)

In [ ]:
all_train_csv.insert(2, 'title_crawling', title)
all_train_csv

,ID,text,title_crawling,target,url,date
0,ynat-v1_train_00000,유튜브 내달 2일까지 크리에이터 지원 공간 운영,출마선언 마치고 질문에 답하는 송영길,3,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=227&oid=001&aid=0008508947,2016.06.30. 오전 10:36
1,ynat-v1_train_00001,어버이날 막따가 흐려저…남부지방 여튼 황사,출마선언 마치고 질문에 답하는 송영길,3,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=103&sid2=248&oid=001&aid=0008384783,2016.05.08. 오전 5:25
2,ynat-v1_train_00002,내년부터 국가RD 평가 때 논문건수는 반영 않는다,출마선언 마치고 질문에 답하는 송영길,2,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=228&oid=001&aid=0008254585,2016.03.15. 오후 12:00
3,ynat-v1_train_00003,김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것,출마선언 마치고 질문에 답하는 송영길,2,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=228&oid=001&aid=0009070646,2017.02.28. 오전 9:54
4,ynat-v1_train_00004,회새긴간 작까 김동시 걍심꼬백 뜽 새 소설집 뚜권 출간,출마선언 마치고 질문에 답하는 송영길,3,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=103&sid2=243&oid=001&aid=0009999529,2018.04.03. 오전 7:05
...,...,...,...,...,...,...
45673,ynat-v1_train_45673,사우디 아람코 주요 석유시설·유전 드론 공격에 화재,출마선언 마치고 질문에 답하는 송영길,4,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=104&sid2=234&oid=001&aid=0011080278,2019.09.14. 오후 4:36
45674,ynat-v1_train_45674,박기원 감독 눈치 보지 말고…비예나 눈치 본 건 아닌데,출마선언 마치고 질문에 답하는 송영길,5,https://sports.news.naver.com/news.nhn?oid=001&aid=0011186865,2019.11.03 17:06
45675,ynat-v1_train_45675,아시안게임 첫 경기 앞둔 라건아 인도네시아 팬이라도 ...,출마선언 마치고 질문에 답하는 송영길,5,https://sports.news.naver.com/news.nhn?oid=001&aid=0010267768,2018.08.13 17:54
45676,ynat-v1_train_45676,트럼프 미중 무역협상 1단계 합의 서명식 가질 것종합,출마선언 마치고 질문에 답하는 송영길,4,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=104&sid2=232&oid=001&aid=0011297568,2019.12.25. 오후 4:36


In [ ]:
none_text_check = all_train_csv['title_crawling'] == 'None'
print(none_text_check.value_counts())

text_diff = all_train_csv[none_text_check]
print(text_diff.shape)
text_diff

In [ ]:
def text_preprocessing(input) :
    if input['title_crawling'] == 'None': 
        return title_crawling(input['url'])

In [ ]:
all_train_csv['title_crawling'] = all_train_csv.apply(text_preprocessing, axis=1)
all_train_csv.head(3)

In [ ]:
clear_g2p_noise = all_train_csv[['ID', 'title_crawling', 'target', 'url', 'date']]
clear_g2p_noise.columns = ['ID', 'text', 'target', 'url', 'date']
clear_g2p_noise.to_csv('/opt/level2_nlp_datacentric-nlp-12/data/clear_g2p_noise_all_train.csv', sep = ',', index=False)